In [7]:
import jieba
from datasets import load_dataset
import random
import nltk
from nltk.corpus import wordnet

In [8]:
def get_synonyms(word: str):
    synsets = wordnet.synsets(word)
    synonyms = set()
    for syn in synsets:
        for lemma in syn.lemmas():
            name = lemma.name().replace("_", " ")
            if name.lower() != word.lower():
                synonyms.add(name)
    return list(synonyms)

def synonym_replacement(sentence: str, n: int = 1) -> str:
    """
    随机选 n 个可替换的单词，用其同义词替换。
    """
    words = sentence.split()
    # 可替换词：非停用词、长度>2 且有同义词
    candidates = [w for w in set(words) if len(get_synonyms(w)) > 0]
    random.shuffle(candidates)
    num_replaced = 0
    for w in candidates:
        syns = get_synonyms(w)
        if syns:
            synonym = random.choice(syns)
            # 全文替换该词的所有出现
            sentence = sentence.replace(w, synonym, 1)
            num_replaced += 1
        if num_replaced >= n:
            break
    return sentence
def random_deletion(words, p=0.1):
    # 每个词以 p 的概率删除
    if len(words) == 1:
        return words
    return [w for w in words if random.random() > p]

def random_swap(words, n_swaps=1):
    words = words.copy()
    n = len(words)
    if n <= 2:
        return words
    for _ in range(n_swaps):
        i, j = random.sample(range(n), 2)
        words[i], words[j] = words[j], words[i]
    return words

def random_insertion(words, n_insert=1):
    new_words = words.copy()
    for _ in range(n_insert):
        candidates = [w for w in new_words if get_synonyms(w)]
        if not candidates: break
        word = random.choice(candidates)
        synonym = random.choice(get_synonyms(word))
        pos = random.randint(0, len(new_words))
        new_words.insert(pos, synonym)
    return new_words

def eda_augment(sentence: str, alpha_sr=0.1, alpha_ri=0.1,
                alpha_rs=0.1, p_rd=0.1, num_aug=1):
    """
    对一句话生成 num_aug 条 EDA 增强样本
    """
    words = list(jieba.cut(sentence))
    augmented = []
    n_sr = max(1, int(alpha_sr * len(words)))
    n_ri = max(1, int(alpha_ri * len(words)))
    n_rs = max(1, int(alpha_rs * len(words)))

    # 1) 同义词替换
    a_words = synonym_replacement(sentence, n_sr).split()
    augmented.append(" ".join(a_words))
    # 2) 随机插入
    a_words = random_insertion(words, n_ri)
    augmented.append(" ".join(a_words))
    # 3) 随机交换
    a_words = random_swap(words, n_rs)
    augmented.append(" ".join(a_words))
    # 4) 随机删除
    a_words = random_deletion(words, p_rd)
    augmented.append(" ".join(a_words))

    # 随机选 num_aug 条返回
    random.shuffle(augmented)
    return augmented[:num_aug]
    
# def augment_examples(example):
#     text = example["text"]
#     # 1) 同义词替换一条
#     sr = synonym_replacement(text, n=2)
#     # 2) EDA 生成两条
#     eda_samples = eda_augment(text, alpha_sr=0.1, p_rd=0.1, num_aug=2)
#     # 返回原文 + 三条增强
#     augmented_texts = [text, sr] + eda_samples
#     return {"text": str(augmented_texts),
#             "label": [example["label"]] * len(augmented_texts)}

In [9]:
from datasets import load_dataset

# 加载原始训练集
ds = load_dataset("json", data_files="train.jsonl", split="train")

In [10]:
# 3. 数据预处理：tokenize
label2id = {"财经": 0, "体育": 1, "娱乐": 2, "教育": 3, "科技": 4}
id2label = {v: k for k, v in label2id.items()}

# 把字符串 label 映射为数字
def encode_labels(example):
    example["label"] = label2id[example["label"]]
    return example
ds = ds.map(encode_labels)

In [11]:
from datasets import Features, Sequence, Value

# 1. 定义新 schema：text 变成一个列表序列
new_features = Features({
    "text": Sequence(feature=Value("string")),     # 现在 text 列是 string 序列
    "label":Sequence(feature=Value("int64")),      # 假设 label 已经是 int
})

def augment_examples(batch):
    # batch["text"] 是 List[str]，batch["label"] 是 List[int]
    out_texts, out_labels = [], []

    for text, label in zip(batch["text"], batch["label"]):
        augmented = eda_augment(text, num_aug=3)  # 生成 3 条增强文本
        # 原本那条也保留一份
        all_texts = [text] + augmented
        out_texts.append(all_texts)
        # 每一条增强都用同一个 label
        out_labels.append([label] * len(all_texts))

    return {"text": out_texts, "label": out_labels}

# 2. 批量 map，指定 features
aug_ds = ds.map(
    augment_examples,
    batched=True,
    batch_size=100,           # 适当调速
    remove_columns=ds.column_names,
    features=new_features,
)

# 3. 扁平化：把 List[List] → flat rows
aug_ds = aug_ds.flatten()

print(aug_ds)
# 每行就变成了单条 text + 对应 label


Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.824 seconds.
Prefix dict has been built successfully.


KeyboardInterrupt: 

In [6]:
print(aug_ds[0])

{'text': ['AC米兰至尊瑰宝身价全意甲第2 巴雷西他令我热血沸腾\n\u3000\u3000新浪体育讯\u3000帕托复出4场，打进5球，而恰恰AC米兰在这4场比赛中取得全胜拿到了12分，而在这4连胜之前，AC米兰则是一负二平，可见帕托对于AC米兰的重要性。\n\u3000\u3000帕托对佛罗伦萨的进球，有人预料到了，因为帕托自加盟意甲以来就是“佛罗伦萨克星”，4战佛罗伦萨，每战必进球，而对亚特兰大的梅开二度却出乎宿命论者预料，登陆意甲之后，帕托还未攻破过亚特兰大的球门，而过去每次面对这个对手，帕托的表现都很糟糕，尤其是07/08赛季对阵亚特兰大之后，帕托一度成为了替补，但现在的帕托似乎已经是神挡杀神佛挡杀佛，面对昔日自己的“苦手”，以两个漂亮的进球成了比赛的决定性人物。就像巴雷西所说：“帕托的表现让AC米兰近几轮的攻击力改观了很多，他是一个绝对的天才，他的技术无与伦比，很难有人达到他的程度。帕托和巴洛特利是年轻球员当中的佼佼者，看他们踢球让我热血沸腾。”\n\u3000\u3000其实从比赛的一开始，帕托就表现出了比前几轮更好的状态，27分钟，他在中路接到贝克汉姆的传球后，转身一领摆脱了帕多因的防守后，塞出直线球给插上的博列洛，可惜博列洛将球带入禁区后，把一对一的机会打偏。31分钟，帕托本场的第一次射门就攻破了对方球门，当时安布罗西尼接到小罗妙传，将球吊向后点，帕托看准来球与球门，张弓搭箭，将身体和右腿拉到和地面平行，面对凌空飞来的皮球停也不停，横身就打！尽管力量不算大，但帕托的动作干净利落一气呵成，球速很快，再加上他有意识将皮球打向地面，尽管本场亚特兰大的门将孔西利做出了不少精彩扑救，但对此球还是无能为力，指尖够了一下之后，皮球仍撞柱飞入网中！\n\u3000\u3000而第二个进球更富戏剧性，帕托这一次又是接到小罗的助攻，反越位成功，横向一拨扣过封堵角度的门将孔西利，尽管曼弗雷迪尼奋力铲断，但由于皮球完全在帕托的控制范围之内，球依然是打到了帕托的脚上，撞进了面前的空门，进球后的帕托，显然对这一颇具戏剧性的得分非常喜悦--这是他本赛季的第12个进球，尽管之前伤停了5轮，在意甲射手榜上，帕托已经上升至第三位，而如果除去点球，单算运动战进球榜，帕托已经与国际米兰的迭戈-米利托并列第二！莱昂纳多赛前曾经表示：“帕托是一名长于一对一的球员，但他同时也是一个得

In [5]:
from datasets import Dataset

texts, labels = [], []
for example in ds:
    orig_text, orig_label = example["text"], example["label"]
    # 原本一条 + 3 条增强
    aug_texts = [orig_text] + eda_augment(orig_text, num_aug=3)
    texts.extend(aug_texts)
    labels.extend([orig_label] * len(aug_texts))

aug_ds = Dataset.from_dict({"text": texts, "label": labels})
print(aug_ds)


Dataset({
    features: ['text', 'label'],
    num_rows: 36000
})


In [ ]:
aug_ds.to_json("THUCNews_augmented.jsonl", orient="records", lines=True, force_ascii=False)